In [13]:
select_example = input('Path or Example:')
FILE_PATH = None
SAMPLES = {
    1: 'KOAX_20240521_1844',
    2: 'KOAX_20240521_1849',
    3: 'KOAX_20240523_2138',

}
if select_example.isdigit():
    FILE_PATH = f"{'./Samples/'}{SAMPLES[int(select_example)]}" # I don't care if it is out of range, I just want to test the code
else:
    FILE_PATH = select_example
import os
fsize = os.stat(FILE_PATH).st_size
f = open(FILE_PATH, 'rb')
import bz2
import namedstruct as ns
import logging

In [14]:
f.seek(0)

log = logging.getLogger(__name__)

VHR = ns.nstruct(
    (ns.char[6],'Tape'),            # Magic number: AR2V00
    (ns.char[2],'Version'),         # Version number
    (ns.char[1],'End of Tape'),     # End of Tape marker
    (ns.char[3],'Sequence Number'), # Extension number
    (ns.uint32,'Date_d'),           # Date of Volume, days since 1/1/1970
    (ns.uint32,'Time_ms'),          # Time of Volume, milliseconds since midnight
    (ns.char[4],'Station'),         # Station ID (ICAO)
    name='VHR',padding=1,endian='>')
LDM_CR = ns.nstruct(
    (ns.int32,'ctrl_word'),      # Control word - lenth of the compressed data
    (ns.raw,'data'),                       # This data is bz2 compressed when saved
    name='LDM_CR',padding=1,endian='>') # TODO: Need to tell it to compress when packing, save the size to the control word


L2A = ns.nstruct(
    (VHR,'VHR'),                       # Volume Header Record
    (LDM_CR[0],'records'),             # LDM_CR records 
    name='L2A',padding=1,endian='>')

# test = VHR.parse(f.read(24))
fi, __ = L2A.parse(f.read())
# go back to the first LDM_CR
f.seek(24)
while(f.tell() < fsize): # read each LDM_CR into a record and add it to the archive - we do this since the datatype is raw here.
    l = f.tell()
    s = int.from_bytes(f.read(4))
    if(s==b'\xff\xff'):
        break
    f.seek(l)
    d = f.read(s+4)
    if (d == b''):
        log.debug('bad data')
    ldm = LDM_CR.create(d)
    fi.records.append(ldm)


In [26]:

if True:
    f.close() # close the file
    log.setLevel(0)
msg_header_channel_enum = ns.enum('msg_header_channel_enum', None, ns.uint8, bitwise=False,
                    # Note: if both redundent channels are 0s, single channel mode. 
                           RC_1 = 0,    # redundant channel 1
                           RC_2 = 1,    # redundant channel 2
                           unused = 2,  # unused. 
                           ORDA = 3     # ORDA channel (vs legacy)
                           )
# Message header structure 
msg_header = ns.nstruct(
        (ns.uint32[3],'prepad'),                # padding
        (ns.uint16,'message_size'),             # size of message in halfwords
        (msg_header_channel_enum,'channel'),    # actually bit field
        (ns.uint8,'message_type'),              # type of message
        (ns.uint16,'sequence'),                 # sequence number of message
        (ns.uint16,'date'),                     # days since 1/1/1970
        (ns.uint32,'milliseconds'),             # milliseconds since midnight
        # Remember, is these 2 are not 1, then the message is extended and needs to be combined with the next message
        (ns.uint16,'segments'),                 # number of segments in message  
        (ns.uint16,'segment'),                  # segment number of this message
        name='msg_header', padding=1, prepack=ns.packvalue(2432,'message_size'), endian='>') # TODO: Use size of message to determine size of message 

# Base message structure
msg_base = ns.nstruct(
    (msg_header,'header'),
    (ns.raw,'data'),
    name='msg_base',padding=1,endian='>')

# UNCOMPRESSED RECORD data
LDM_UR = ns.nstruct(
    (msg_base[0],'msg'),                # Messages - not sure that this will work since it will hold extended data
    name='LDM_UR',padding=1,endian='>') # TODO: Need to tell it to compress when packing, save the size to the control word

RECORD_HOLDER = ns.nstruct(
    (LDM_UR[0],'ldm'),
    name='RECORD_HOLDER',padding=1,endian='>')

# log.setLevel(logging.FATAL-1)
record_count = len(fi.records)
print(f"Number of records: {record_count}")

record = 0
records = RECORD_HOLDER.create('')
for cr in fi.records:
    if(cr.ctrl_word < 0):
        break
    data = bz2.decompress(cr.data[:cr.ctrl_word])
    if data == b'':
        log.error(f"Error decompressing data. CW: {cr.ctrl_word}")
        break


    position = 0 # position in the data
    i = 0 # message counter
    ldm = LDM_UR.create('')
    while(position<len(data)):
        msg_size = int.from_bytes(data[position+12:position+14]) # get size in halfwords
        if(msg_size > 65535):
            log.error(f"Error: Message size too large: {msg_size}")
            raise Exception(f"Stop {msg_size}")
            break
        elif(msg_size<9):
            if (msg_size != 0):
                log.warning(f"Error?: Message size too small: {msg_size}")
                raise Exception(f"Stop {record}.{i}")
            else:
                log.info(f"Message {i} is empty")
                # break
        else:
            pass

        if(record == 0):
            eom = 2432*(i+1)-12 # not really needed but..
        else:
            eom = position+msg_size*2
        
        dat = data[position:eom]
        if dat[0:12] != b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00':
            log.error(f"Error: Prepad {i} is not 0: {dat[0:12]}")
        elif (dat[12:14] == b'\x00\x00' or dat[12:14] == b''):
            log.info(f"in {i}: Product is 0s: {dat[0:28]}")
        else:
            pass
        #TODO: parse the message into the correct message type instead of raw data
        try:
            a_record_that_exists = msg_base.create(dat) 
            ldm.msg.append(a_record_that_exists)
        except Exception as e:
            log.error(f"Error: {e} -  {dat[0:28]}")
            break
        position=eom+12
        i+=1
    records.ldm.append(ldm)
    record+=1
    # log.fatal(f"going to next record {record}")

# log.setLevel(1000)
prods = {}
totals = 0
for i, ldm in enumerate(records.ldm):
    for j, a in enumerate(ldm.msg):
        if a.header.message_type > 33:
            log.warning(f"Message {i}.{j} type {a.header.message_type} is not valid!")
        if a.header.message_type == 0:
            log.info(f"Message {i}.{j} has type 0. Likley a type 13 legacy area.")
        if a.header.milliseconds >= 86400000:
            log.warning(f"Message {i}.{j} time is invalid: {a.header.milliseconds}")
        if a.header.segments != 1 or a.header.segment != 1:
            log.warning(f"Message {i}.{j} has segments: {a.header.segment} of {a.header.segments}")
        if a.header.date > 65535:
            log.warning(f"Message {i}.{j} date is invalid: {a.header.date}")
        if a.header.sequence > 65535:
            log.warning(f"Message {i}.{j} sequence is invalid: {a.header.sequence}")
        if a.header.channel > 10:
            log.warning(f"Message {i}.{j} redundent channel is invalid: {a.header.channel}")
        if a.header.message_size > 65535:
            log.warning(f"Message {i}.{j} size is invalid: {a.header.message_size}")
        if a.header.prepad[0] != 0 or a.header.prepad[1] != 0 or a.header.prepad[2] != 0:
            log.warning(f"Message {i}.{j} Prepad is not 0: {a.header.prepad}")
        # log.warning(f"...Message {i}.{j}: {a.header.message_type} has sequence number {a.header.sequence} and is {a.header.message_size} halfwords")
        prods[a.header.message_type] = prods.get(a.header.message_type,0)+1
        totals+=1
print(prods)
print(totals)
# 3.2.4.17


Number of records: 94


Error: Prepad 105 is not 0: b''
Error: Cannot parse struct: data is corrupted. -  b''
Error: Prepad 14 is not 0: b''
Error: Cannot parse struct: data is corrupted. -  b''
Message 0.0 has segments: 1 of 5
Message 0.1 has segments: 2 of 5
Message 0.2 has segments: 3 of 5
Message 0.3 has segments: 4 of 5
Message 0.4 has segments: 5 of 5
Message 0.5 has segments: 0 of 0
Message 0.6 has segments: 0 of 0
Message 0.7 has segments: 0 of 0
Message 0.8 has segments: 0 of 0
Message 0.9 has segments: 0 of 0
Message 0.10 has segments: 0 of 0
Message 0.11 has segments: 0 of 0
Message 0.12 has segments: 0 of 0
Message 0.13 has segments: 0 of 0
Message 0.14 has segments: 0 of 0
Message 0.15 has segments: 0 of 0
Message 0.16 has segments: 0 of 0
Message 0.17 has segments: 0 of 0
Message 0.18 has segments: 0 of 0
Message 0.19 has segments: 0 of 0
Message 0.20 has segments: 0 of 0
Message 0.21 has segments: 0 of 0
Message 0.22 has segments: 0 of 0
Message 0.23 has segments: 0 of 0
Message 0.24 has segmen

{15: 5, 0: 122, 18: 4, 3: 1, 5: 1, 2: 3, 31: 10917}
11053


# Expected Blocks:
## Metadata
1. Message 15 - up to 77 Segments
2. Message 13 - up to 49 Segments - WIll be 0s after build 19
3. Message 18 - 5 Segments
4. Message 3 - 1 Segment
5. Message 5 - 1 Segment
6. Message 2 - 1 Segment
TOTAL: 134 Segments/Messagges
## Data 
- Type 31 * 120 radials
- Type 2 * (0+)
## Model Data 
- Type 29